In [4]:
from collections import Counter
import sklearn
import random
import pandas as pd
from itertools import chain
from sklearn.model_selection import StratifiedKFold

from pandas import read_csv
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import numpy as np
from sklearn.base import is_regressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble.forest import BaseForest
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from sklearn.tree.tree import BaseDecisionTree
from catboost import Pool, CatBoostClassifier
from scipy.stats.mstats import gmean
from random import randint
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import os
from pycm import *
import statistics
from scipy.stats.mstats import gmean

#############################################################################
#############################################################################
#############################################################################

dataframe=read_csv('18.csv')

dim=dataframe.shape
array = dataframe.values    
Y = array[:,dim[1]-1]
X = array[:,0:dim[1]-1]

minClass=10000000;
minClass_idx=0;
from collections import Counter
s=Counter(Y)
for idx, val in enumerate(s):
    if(s[val]<minClass):
        minClass=s[val];
        minClass_idx=val;

        
j=0
SEED=42

AUCNU=list()
AUCNU_Whole=list()
Accuracy=list()
Accuracy_Whole=list()
Precision=list()
Precision_Whole=list()
F1=list()
F1_Whole=list()
GM=list()
GM_Whole=list()
gm_temp=list()
TPR=list()
TPR_Whole=list()
TNR=list()
TNR_Whole=list()  #Specifity
tnr_temp=list()
G=list()
G_Whole=list()
g_temp=list()
Kappa=list()   #Kappa not computed for each class - It is an overall measurement
Kappa_Whole=list() 
gr_temp = list()
GR=list()
GR_Whole=list()
Recall_MinorityClass=list()
Recall_MinorityClass_Whole=list()
Precision_MinorityClass=list()
Precision_MinorityClass_Whole=list()
F_MinorityClass=list()
F_MinorityClass_Whole=list()
temp_mauc=list()
MAUC=list()
MAUC_Whole=list()
CEN=list()
CEN_Whole=list()
MCC=list()
MCC_Whole=list()
temp_mcc_ovo=list()
MCC_OVO=list()
MCC_OVO_Whole=list()
temp_mcc_ova=list()
MCC_OVA=list()
MCC_OVA_Whole=list()


minClass=10000000;
minClass_idx=0;
from collections import Counter
s=Counter(Y)
for idx, val in enumerate(s):
    if(s[val]<minClass):
        minClass=s[val];
        minClass_idx=val;

for k in range(10):
    SEED=randint(2, 10);
    for i in range(5):
    #for train_index, test_index in skf.split(X,Y): 
        X_train, X_test, y_train, Y_test = train_test_split(X, Y, test_size=.20, stratify=Y,random_state=i*SEED)

        # train model and make predictions
        classification = CatBoostClassifier(iterations=100, learning_rate=0.3, depth=5, loss_function='MultiClass')
        classification.fit(X_train, y_train) 
        y_preds = classification.predict(X_test) 
        y_preds = list(chain.from_iterable(y_preds))
        
        y_preds=[int(i) for i in y_preds]
        y_preds = np.asarray(y_preds)
        cm = ConfusionMatrix(actual_vector=Y_test, predict_vector=y_preds) 
        cm.save_html(os.path.join("","CatBoost_Run="+str(k)+"_Fold="+str(i)))
        
        Test_ClassSet=list(set(Y_test))
        ova_test=Y_test
        ova_pred=y_preds
        for ind1_c in range(0,len(Test_ClassSet)):
            ova_index1=[ind for ind, x in enumerate(Y_test) if x == Test_ClassSet[ind1_c]]
            ova_index2=[ind for ind, x in enumerate(Y_test) if x != Test_ClassSet[ind1_c]]
            ova_test[ova_index1]=1
            ova_test[ova_index2]=-1
            ova_pred[ova_index1]=1
            ova_pred[ova_index2]=-1
            ova_pred[ova_index2]=0
            temp_mcc_ova.append(matthews_corrcoef(ova_test, ova_pred))
            for ind2_c in range(ind1_c+1,len(Test_ClassSet)):
                Class1_index=[ind for ind, x in enumerate(Y_test) if x == Test_ClassSet[ind1_c]]  
                Class2_index=[ind for ind, x in enumerate(Y_test) if x == Test_ClassSet[ind2_c]]  
                test1=Y_test[Class1_index]
                test2=Y_test[Class2_index]
                pred1=y_preds[Class1_index]
                pred2=y_preds[Class2_index]
                test_classes=np.concatenate((test1,test2), axis=0)
                pred_classes=np.concatenate((pred1,pred2), axis=0) 
                for qq in range(0, len(test_classes)):
                    if(test_classes[qq]!=pred_classes[qq] and test_classes[qq]==Test_ClassSet[ind1_c]):
                        pred_classes[qq]=Test_ClassSet[ind2_c]
                    elif(test_classes[qq]!=pred_classes[qq] and test_classes[qq]==Test_ClassSet[ind2_c]):
                        pred_classes[qq]=Test_ClassSet[ind1_c] 
                temp_mauc.append(roc_auc_score(test_classes, pred_classes))
                temp_mcc_ovo.append(matthews_corrcoef(test_classes, pred_classes))
                
        for idx, tt in enumerate(cm.F1):            
            gm_temp.append(cm.GM[tt])
            tnr_temp.append(cm.TNR[tt])                       
            g_temp.append(cm.G[tt])
            gr_temp.append(cm.TPR[tt])
            
        
        gm_temp=[0 if v is 'None' else v for v in gm_temp]
        g_temp=[0 if v is 'None' else v for v in g_temp]
        gr_temp=[0 if v is 'None' else v for v in gr_temp]
        tnr_temp=[0 if v is 'None' else v for v in tnr_temp]
        MCC_OVA.append(statistics.mean(temp_mcc_ova))
        temp_mcc_ova.clear()
        MCC_OVO.append(statistics.mean(temp_mcc_ovo))
        temp_mcc_ovo.clear()
        GM.append(statistics.mean(gm_temp))        
        GR.append(gmean(gr_temp))        
        gr_temp.clear()
        G.append(statistics.mean(g_temp))
        g_temp.clear()
        Kappa.append(cm.Kappa)
        AUCNU.append(cm.AUNU)
        F1.append(cm.F1_Macro)
        Precision.append(cm.PPV_Macro)
        TPR.append(cm.TPR_Macro)
        Accuracy.append(cm.Overall_ACC)
        tnr_temp=[0 if v is 'None' else v for v in tnr_temp]
        TNR.append(statistics.mean(tnr_temp))  
        tnr_temp.clear()
        Recall_MinorityClass.append(cm.TPR[minClass_idx])
        Precision_MinorityClass.append(cm.PPV[minClass_idx])
        F_MinorityClass.append(cm.F1[minClass_idx])
        MAUC.append(statistics.mean(temp_mauc))
        temp_mauc.clear()        
        MCC.append(cm.Overall_MCC)
        CEN.append(cm.Overall_CEN)
    
    AUCNU=[0 if v is 'None' else v for v in AUCNU]
    AUCNU_Whole.append(statistics.mean(AUCNU))
    MAUC_Whole.append(statistics.mean(MAUC))
    MAUC.clear()
    AUCNU.clear()    
    Accuracy_Whole.append(statistics.mean(Accuracy))
    Accuracy.clear()
    Precision=[0 if v is 'None' else v for v in Precision]
    Precision_Whole.append(statistics.mean(Precision))
    Precision.clear()     
    F1_Whole.append(statistics.mean(F1))
    F1.clear()
    GM=[0 if v is 'None' else v for v in GM]
    GM_Whole.append(statistics.mean(GM))
    GM.clear()
    TPR=[0 if v is 'None' else v for v in TPR]
    TPR_Whole.append(statistics.mean(TPR))
    TPR.clear() 
    TNR=[0 if v is 'None' else v for v in TNR]
    TNR_Whole.append(statistics.mean(TNR))
    TNR.clear()
    G=[0 if v is 'None' else v for v in G]
    G_Whole.append(statistics.mean(G))
    G.clear()
    Kappa=[0 if v is 'None' else v for v in Kappa]
    Kappa_Whole.append(statistics.mean(Kappa))
    Kappa.clear()
    GR_Whole.append(statistics.mean(GR))
    GR.clear()
    Recall_MinorityClass=[0 if v is 'None' else v for v in Recall_MinorityClass]
    Recall_MinorityClass_Whole.append(statistics.mean(Recall_MinorityClass))
    Precision_MinorityClass=[0 if v is 'None' else v for v in Precision_MinorityClass]
    Precision_MinorityClass_Whole.append(statistics.mean(Precision_MinorityClass))
    F_MinorityClass_Whole.append(statistics.mean(F_MinorityClass))
    Recall_MinorityClass.clear()
    Precision_MinorityClass.clear()
    F_MinorityClass.clear()
    MCC_Whole.append(statistics.mean(MCC))
    MCC.clear()
    CEN_Whole.append(statistics.mean(CEN))
    CEN.clear()
    MCC_OVO_Whole.append(statistics.mean(MCC_OVO))
    MCC_OVO.clear()
    MCC_OVA_Whole.append(statistics.mean(MCC_OVA))
    MCC_OVA.clear()
    

with open('CATBoost.txt', 'w') as filehandle:
                filehandle.write('AUCNU')
        filehandle.write('\n')
        for listitem in AUCNU_Whole:
            filehandle.write('%s\t' % listitem)   
        filehandle.write('\n\n')
        filehandle.write('MAUC')
        filehandle.write('\n')
        for listitem in MAUC_Whole:
            filehandle.write('%s\t' % listitem) 
        #filehandle.write('\n\n')
        #filehandle.write('ACCURACY')
        #filehandle.write('\n')
        #for listitem in Accuracy_Whole:
            #filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('Precision')
        #filehandle.write('\n')
        #for listitem in Precision_Whole:
            #filehandle.write('%s\t' % listitem)
        filehandle.write('\n\n')
        filehandle.write('F_measure'),
        filehandle.write('\n')
        for listitem in F1_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n\n')
        filehandle.write('MCC_measure'),
        filehandle.write('\n')
        for listitem in MCC_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n\n')
        filehandle.write('CEN_measure'),
        filehandle.write('\n')
        for listitem in CEN_Whole:
            filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('G_mean')
        #filehandle.write('\n')
        #for listitem in GM_Whole:
            #filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('TPR')
        #filehandle.write('\n')
        #for listitem in TPR_Whole:
            #filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('TNR')
        #filehandle.write('\n')
        #for listitem in TNR_Whole:
            #filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('G_measure')
        #filehandle.write('\n')
        #for listitem in G_Whole:
            #filehandle.write('%s\t' % listitem)
        filehandle.write('\n\n')
        filehandle.write('Kappa')
        filehandle.write('\n')
        for listitem in Kappa_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('GR')
        filehandle.write('\n')
        for listitem in GR_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('Recall_MinorityClass')
        filehandle.write('\n')
        for listitem in Recall_MinorityClass_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('Precision_MinorityClass')
        filehandle.write('\n')
        for listitem in Precision_MinorityClass_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('F_MinorityClass')
        filehandle.write('\n')
        for listitem in F_MinorityClass_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('MCC_OVO')
        filehandle.write('\n')
        for listitem in MCC_OVO_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('\n')
        filehandle.write('MCC_OVA')
        filehandle.write('\n')
        for listitem in MCC_OVA_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')

        
        
print("%.4f" % statistics.mean(AUCNU_Whole)) #AUC_NU=
print("%.4f" % statistics.mean(MAUC_Whole)) #MAUC
#print("%.4f" % statistics.mean(Accuracy_Whole))
#print("%.4f" % statistics.mean(Precision_Whole))
print("%.4f" % statistics.mean(F1_Whole))
#print("%.4f" % statistics.mean(GM_Whole))
#print("%.4f" % statistics.mean(TPR_Whole)) #Recall
#print("%.4f" % statistics.mean(TNR_Whole)) #Specifity
#print("%.4f" % statistics.mean(G_Whole))
print("%.4f" % statistics.mean(MCC_Whole))
print("%.4f" % statistics.mean(CEN_Whole))
print("%.4f" % statistics.mean(Kappa_Whole))
print("%.4f" % statistics.mean(GR_Whole))
print("%.4f" % statistics.mean(Recall_MinorityClass_Whole))
print("%.4f" % statistics.mean(Precision_MinorityClass_Whole))
print("%.4f" % statistics.mean(F_MinorityClass_Whole))
print("%.4f" % statistics.mean(MCC_OVO_Whole))
print("%.4f" % statistics.mean(MCC_OVA_Whole))
MCC_OVA_Whole.clear()
MCC_Whole.clear()
MAUC_Whole.clear()
F1_Whole.clear
MAUC_Whole.clear()

0:	learn: 1.5746704	total: 19.9ms	remaining: 1.97s
1:	learn: 1.2353514	total: 33.4ms	remaining: 1.64s
2:	learn: 1.0259888	total: 47.2ms	remaining: 1.52s
3:	learn: 0.9031659	total: 60.6ms	remaining: 1.46s
4:	learn: 0.7842157	total: 73.2ms	remaining: 1.39s
5:	learn: 0.6925290	total: 89ms	remaining: 1.39s
6:	learn: 0.6136226	total: 110ms	remaining: 1.46s
7:	learn: 0.5547324	total: 124ms	remaining: 1.43s
8:	learn: 0.4961170	total: 148ms	remaining: 1.49s
9:	learn: 0.4489593	total: 164ms	remaining: 1.47s
10:	learn: 0.4083151	total: 185ms	remaining: 1.49s
11:	learn: 0.3770753	total: 203ms	remaining: 1.49s
12:	learn: 0.3544301	total: 263ms	remaining: 1.76s
13:	learn: 0.3297537	total: 301ms	remaining: 1.85s
14:	learn: 0.3030391	total: 339ms	remaining: 1.92s
15:	learn: 0.2841183	total: 377ms	remaining: 1.98s
16:	learn: 0.2668319	total: 393ms	remaining: 1.92s
17:	learn: 0.2518000	total: 435ms	remaining: 1.98s
18:	learn: 0.2395977	total: 456ms	remaining: 1.95s
19:	learn: 0.2278241	total: 477ms	rem

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:308: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))


KeyError: 6